## 1. Preprocesamiento de Datos

### Conversión de variables categóricas:
- Convertir directamente las variables categóricas a numéricas.
- Utilizar **Label Encoding** o **OneHotEncoding**.
- Representar las variables categóricas utilizando la probabilidad de la clase objetivo.

### Imputación de valores faltantes:
- Imputar los valores faltantes utilizando el valor **0**.
- Imputar los valores faltantes utilizando la **media** para variables numéricas.


In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold


In [ ]:
# Cargar los datos
train_path = '/home/alvaro/Documentos/DATA/titanic/train.csv'
test_path = '/home/alvaro/Documentos/DATA/titanic/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# ======== PREPROCESAMIENTO DE TRAIN ========

# Separar la variable objetivo
supervivientes_col = train["Survived"].copy()

# Eliminar columnas innecesarias
train.drop(columns=["Survived", "Cabin", "Name", "Ticket"], inplace=True)

# Imputadores entrenados con TRAIN
num_cols = train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = train.select_dtypes(include='object').columns

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')

train[num_cols] = num_imputer.fit_transform(train[num_cols])
train[cat_cols] = cat_imputer.fit_transform(train[cat_cols])

# One-hot encoding
cols_to_encode = ['Sex', 'Embarked']
train = pd.get_dummies(train, columns=cols_to_encode)

# Guardamos las columnas resultantes para replicar en test
train_columns = train.columns

# Volver a añadir la variable objetivo
train["Survived"] = supervivientes_col



# ======== PREPROCESAMIENTO DE TEST ========

# Eliminar las mismas columnas
test.drop(columns=["Cabin", "Name", "Ticket"], inplace=True)

# Asegurar que estén las mismas columnas para imputar
test[num_cols] = num_imputer.transform(test[num_cols])
test[cat_cols] = cat_imputer.transform(test[cat_cols])

# One-hot encoding
test = pd.get_dummies(test, columns=cols_to_encode)

# Añadir columnas faltantes (por si test no tiene todas las categorías)
for col in train_columns:
    if col not in test.columns and col != "Survived":
        test[col] = 0

# Reordenar columnas para que coincidan
test = test[[col for col in train_columns if col != "Survived"]]



# Listo para usar en predicciones
print("Train shape:", train.shape)
print("Test shape:", test.shape)


Train shape: (891, 13)
Test shape: (418, 12)


In [48]:
train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_missing,Survived
0,1.0,3.0,22.000000,1.0,0.0,7.2500,False,True,False,False,True,False,0
1,2.0,1.0,38.000000,1.0,0.0,71.2833,True,False,True,False,False,False,1
2,3.0,3.0,26.000000,0.0,0.0,7.9250,True,False,False,False,True,False,1
3,4.0,1.0,35.000000,1.0,0.0,53.1000,True,False,False,False,True,False,1
4,5.0,3.0,35.000000,0.0,0.0,8.0500,False,True,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887.0,2.0,27.000000,0.0,0.0,13.0000,False,True,False,False,True,False,0
887,888.0,1.0,19.000000,0.0,0.0,30.0000,True,False,False,False,True,False,1
888,889.0,3.0,29.699118,1.0,2.0,23.4500,True,False,False,False,True,False,0
889,890.0,1.0,26.000000,0.0,0.0,30.0000,False,True,True,False,False,False,1


## 2. Selección de Características

Explorar diferentes métodos de selección de características:

- Filtrado hi-cuadrado.
- Embebido RFE.
- Envolvente regresión logística.


In [49]:
# CHI2

# X: todas las columnas menos la target
X = train.drop(columns=["Survived"])
y = train["Survived"]

# Selección con Chi-cuadrado
selector_chi2 = SelectKBest(score_func=chi2, k=5)  # elige las 10 mejores
X_chi2 = selector_chi2.fit_transform(X, y)

# Ver columnas seleccionadas
selected_cols_chi2 = X.columns[selector_chi2.get_support()]
print("Características seleccionadas con Chi-cuadrado:")
print(selected_cols_chi2)


Características seleccionadas con Chi-cuadrado:
Index(['Pclass', 'Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')


In [50]:
# RFE

# Usamos un Random Forest como modelo base
model = RandomForestClassifier(random_state=42)
selector_rfe = RFE(estimator=model, n_features_to_select=5)  # elige 10 features

X_rfe = selector_rfe.fit_transform(X, y)
selected_cols_rfe = X.columns[selector_rfe.get_support()]
print("Características seleccionadas con RFE:")
print(selected_cols_rfe)


Características seleccionadas con RFE:
Index(['PassengerId', 'Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')


In [52]:
# Envolvente Regresion Logistica

# Modelo base: regresión logística
logreg = LogisticRegression(max_iter=2000, solver='liblinear')


# Cross-validation
cv = StratifiedKFold(n_splits=5)

selector_rfecv = RFECV(estimator=logreg, step=1, cv=cv, scoring='accuracy')
X_rfecv = selector_rfecv.fit_transform(X, y)

selected_cols_rfecv = X.columns[selector_rfecv.get_support()]
print("Características seleccionadas con RFECV (envolvente con regresión logística):")
print(selected_cols_rfecv)


Características seleccionadas con RFECV (envolvente con regresión logística):
Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Embarked_missing'],
      dtype='object')


## 3. Evaluación de Modelos con Validación Cruzada
Evaluar para la combinación de cada una de las estrategias de codificación
de categorías, imputación y selección de características

- Implementar al menos **dos modelos predictivos**, por ejemplo:
  - **Random Forest**
  - **XGBoost**

- Utilizar **validación cruzada de 10 folds** para evaluar el rendimiento de cada modelo.

- Obtener las siguientes **métricas de desempeño**:
  - Precisión
  - Exactitud
  - Recall
  - F1-Score


## 4. Almacenamiento de Resultados en MySQL

Diseñar y crear una base de datos en **MySQL** para almacenar los resultados de las evaluaciones.  
Se recomienda incluir las siguientes tablas:

- **Información del preprocesamiento**: (estrategia de imputación, codificación utilizada, etc.).

- **Métricas de desempeño por modelo y por fold**  

- **Resumen general de la evaluación**: (por ejemplo, promedio y desviación estándar de las métricas).
